<p style="text-align:center;">
<img src=https://noodle.digitalfutures.com/studentuploads/Data_Cygnets_logo.png width = 150px, height=150px/
     style="float: center; " />
</p>

# Neural Networks 🧠 on Swan Teleco
Trying Neural Network algorithms-Results were worse than a logistic regression model.
### by Data Cygnets
🦢 Jamie M

🦢 Muqadas

🦢 Sennan

🦢 Maarja

## 1. Python Essentials 📁

**Note, some packages are outside of the typical python library**

In [ ]:
#Basic Python packages
from time import time
import warnings
warnings.filterwarnings('ignore')


#Numerical information
import numpy as np

#General Data use
import pandas as pd
from collections import Counter

#Data Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

##Predictive Modelling##
#Metrics
from sklearn import metrics
from sklearn.metrics import (confusion_matrix, accuracy_score)

#Model
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

#HyperParameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split

from sklearn.preprocessing import MinMaxScaler


#Modelling Visuals
from sklearn import tree


#######External packages########
#Smarter Decision Tree Visualisation
#Please run this if the package is not currently installed in the environment:
#!pip install supertree -U
from supertree import SuperTree

#Class balancing with SMOTE(Synthetic Minority Over-sampling Technique-https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/)
#https://medium.com/data-science/the-right-way-of-using-smote-with-cross-validation-92a8d09d00c7#:~:text=We%20first%20split%20the%20data,cross%2Dvalidation%20and%20test%20scores.
#Please run this if the package is not currently installed in the environment:
#!pip install imbalanced-learn -U
import imblearn
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_excel('1 - Project Data.xlsx')

## 2. Data Cleaning and Splitting 📊

### Outlier removal

In [ ]:
df_tenured = df[df['Tenure Months']!=0]

### Data Splitting

In [ ]:
train, test = train_test_split(df_tenured, random_state = 60, stratify = df_tenured['Churn Label'], test_size = 0.2)

## 3. Feature Engineering🛠️

**Features to deal with**
 - CustomerID; Unique Value, only needed for the use of calling the customer. **DROP**
 - Count; Integer Value of 1, only checking that they were/are a customer. **DROP**
 - Country; United States, all within the same country. **DROP**
 - State; California, all within the same state. **DROP**
 - City; Could be useful, does seem to be a higher Churn average is larger cities.
 - Zip Code; Provides the area, more information than the city.

 - Lat Long; Exact location of customer (data risk) **DROP**
 - Latitude; Exact location of customer (data risk) **DROP**
 - Longitude; Exact location of customer (data risk) **DROP**
 - Gender; M/F, Need to change to 0/1
 - Senior Citizen; Y/N, Need to change to 0/1
 - Partner; Y/N, Need to change to 0/1
 - Dependents; Y/N, Need to change to 0/1
 - Tenure Months; Length of Tenure for a customer. Do we need it >n months?
 - Phone Service; Y/N, Need to change to 0/1
 - Multiple Lines; Y/N/No Phone Service, Need to change to 0/1 **Could be changed**
 - Internet Service; DSL/Fiber Optic/No. Need to convert to categorical values
 - Online Security; Y/N/No internet service, Need to change to 0/1 **Could be changed**
 - Online Backup; Y/N/No internet service, Need to change to 0/1 **Could be changed**
 - Device Protection; Y/N/No internet service, Need to change to 0/1 **Could be changed**
 - Tech Support; Y/N/No internet service, Need to change to 0/1 **Could be changed**
 - Streaming TV; Y/N/No internet service, Need to change to 0/1 **Could be changed**
 - Streaming Movies; Y/N/No internet service, Need to change to 0/1 **Could be changed**
 - Contract; M2M/1yr/2yr. Need to convert to categorical values
 - Paperless Billing; Y/N, Need to change to 0/1
 - Payment Method; Mailed check/Electronic check/Bank transfer (automatic)/Credit card (automatic). Need to convert to categorical values
 - Monthly Charges; Numerical Values
 - Total Charges; Numerical Values

**Target(s)**
 - Churn Label; Y/N
 - Churn Value; 1/0


 - Churn Reason; A variety of reasons. Make a model for looking at the most important features for each reason?

In [ ]:
Y_N_cols = ['Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 'Internet Service',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Paperless Billing']
categorical_cols = ['City','Contract','Payment Method']
def str_to_int_boolean(x):
    if x=='Yes':
        return 1
    else:
        return 0
def service_count(row):
    return (~row.str.contains('No')).sum()


def feature_eng(df):
    
    
    #Removal of Columns
    features = list(df.columns)
    #Dropping unnecessary columns#
    features.remove('CustomerID')
    features.remove('Count')
    features.remove('Country')
    features.remove('State')
    features.remove('Lat Long')
    features.remove('Latitude')
    features.remove('Longitude')
    features.remove('Churn Reason')

    #Duplication
    features.remove('Churn Value')
    
    df_copy = df[features].copy()
    df_copy.index = df.CustomerID
    ##Cleaning of Data##
    df_copy.replace('No internet service', 'No', inplace=True)
    df_copy.replace('No phone service', 'No', inplace=True)
    df_copy['Total Charges'] = df_copy['Total Charges'].replace(' ', 0)
    df_copy['Total Charges'] = df_copy['Total Charges'].astype(float)
    
    
    cols = ['Phone Service','Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies']

    #Counting the number of services
    df_copy['service_count'] = df_copy[cols].apply(service_count, axis=1)

    ##Changing from str to integer booleans##
    for i in Y_N_cols:
        df_copy[i] = df_copy[i].apply(str_to_int_boolean)

    df_copy['Gender'] = [1 if val == 'Male' else 0 for val in df_copy['Gender']]

    ##Changing categorical into numerical

    for i in categorical_cols:
        df_copy[i] = pd.factorize(df_copy[i])[0]

    ##Space for further feature Engineering##
    
    
    
    
    y = df_copy['Churn Label'].copy()
    X = df_copy.drop(columns = ['Churn Label']).copy()

    
    return X,y

In [ ]:
X_train_fe, y_train_fe = feature_eng(train)

In [ ]:
X_train_fe.head()

## 4. Model Building and Training🏗️

**SMOTE has improved the recall of the model by ~16%, sacrifising the accuracy by 1-2%**

- MinMaxScaler has been used as neural networks require all values in the range 0-1

**Hyperparameter selections**
- Model alpha: The learning rate of the neural network
- Model hidden layer sizes: The size of each layer that the model trains the data on

In [ ]:
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state=60)],
                                        ['scaler', MinMaxScaler(feature_range=(-1, 1))],
                                        ['classifier', MLPClassifier(solver='adam', alpha=1e-5,
                                                hidden_layer_sizes=(5, 2), random_state=60, max_iter=500)]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=60)
    
param_grid = {'classifier__alpha':[1e-5,1e-6,5e-6], 'classifier__hidden_layer_sizes':[(5,),(10,),(10,5),(25,)]}
grid_search = GridSearchCV(estimator=pipeline,
                            param_grid=param_grid,
                            scoring='accuracy',
                            cv=stratified_kfold,
                            n_jobs=-1)
grid_search.fit(X_train_fe, y_train_fe)



In [ ]:
dt = grid_search.best_estimator_
dt.fit(X_train_fe, y_train_fe)

## 5. Model Prediction using the best hyperparameters 🔮

In [ ]:
X_test_fe, y_test_fe = feature_eng(test)

In [ ]:
print(f'Score on training set: {dt.score(X_train_fe, y_train_fe)}')
print(f'Score on testing set: {dt.score(X_test_fe, y_test_fe)}')

In [ ]:
train_results = X_train_fe.copy()
train_results['y_pred'] = dt.predict(X_train_fe)
train_results['y_real'] = y_train_fe
train_results['y_prob'] = dt.predict_proba(X_train_fe)[:,1]

In [ ]:
test_results = X_test_fe.copy()
test_results['y_pred'] = dt.predict(X_test_fe)
test_results['y_real'] = y_test_fe
test_results['y_prob'] = dt.predict_proba(X_test_fe)[:,1]

### Results on the Training Set

- Accuracy-81%
- Recall on desired class (Yes)-0.68

In [ ]:
print(metrics.confusion_matrix(train_results['y_real'], train_results['y_pred']))
print(metrics.classification_report(train_results['y_real'], train_results['y_pred']))

### Results on the Testing Set

- Accuracy-78%
- Recall on desired class (Yes)-0.58

With this being low, it is often disregarding the churners. This is the avenue we want to maximise.

In [ ]:
print(metrics.confusion_matrix(test_results['y_real'], test_results['y_pred']))
print(metrics.classification_report(test_results['y_real'], test_results['y_pred']))

# 6. Insights from Model 🤯

- The neural network model does not produce a high accuracy no the training nor test set.
- The recall of the testing set largely drops in the desired class.
- The model is slightly overfitting.

**Areas to explore in the future**

- Observing different selections of features.
- Creating a deeper model to further understand the data.